<a href="https://colab.research.google.com/github/angelrodriguez0617/CS-4710-Machine-Learning-II/blob/main/Diffusion_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adversarial Diffusion Distillation (ADD) Model Architecture

The ADD model architecture combines adversarial training with knowledge distillation to efficiently generate high-quality images.

## Components

### ADD-Student Network
- Acts as a denoiser, generating denoised samples from diffused input images.
- Initialized from a pre-trained U-Net-based diffusion model with weights `θ`.

### Adversarial Loss
- Aims to generate samples that a discriminator cannot distinguish from real images.
- The discriminator with trainable weights `ϕ` differentiates between generated and real images.
- Hinge loss function is used for the adversarial objective.

### Discriminator Network
- Utilizes trainable weights to evaluate generated images.
- Employs a pre-trained feature network, such as vision transformers, and lightweight discriminator heads.

### Distillation Loss
- Leverages a pre-trained diffusion model (DM teacher) with frozen weights `ψ`.
- The ADD-student's generated samples are diffused and compared to the teacher's denoising predictions.


The architecture enables the ADD model to produce high-quality images rapidly, leveraging the strengths of adversarial networks and diffusion models for real-time image generation applications.




In [ ]:
# Suppressing warnings to prevent them from displaying in the output
import warnings
warnings.filterwarnings("ignore")

# Installing necessary libraries for generative diffusion models
# This command installs 'diffusers', 'transformers', 'accelerate', and 'mediapy'
%pip install --quiet --upgrade diffusers transformers accelerate mediapy

# Importing required modules for the demonstration
import mediapy as media  # Used for displaying images
import random            # Used for generating random seeds
import sys               # Used for system-specific parameters
import torch             # PyTorch library, fundamental for the model
from diffusers import AutoPipelineForText2Image  # Importing the pipeline for text-to-image conversion

# Load the generative diffusion model with predefined settings
pipe = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/sdxl-turbo",       # Model name
    torch_dtype=torch.float16,      # Data type setting for efficient computation
    use_safetensors=True,           # Ensuring safe tensor operations
    variant="fp16",                 # Model variant
)
pipe = pipe.to("cuda")  # Moving the pipeline to CUDA for GPU usage

# Continuously asking for prompts and generating images
while True:
    prompt = input("Enter your prompt or type 'exit' to stop: ")  # User input for the prompt
    if prompt.lower() == 'exit':
        break  # Break the loop if 'exit' is entered

    seed = random.randint(0, sys.maxsize)  # Generating a random seed
    num_inference_steps = 4  # Setting the number of diffusion steps

    # Generating the image based on the provided prompt
    images = pipe(
        prompt=prompt,  # The text prompt for image generation
        guidance_scale=0.0,  # Guidance scale for generation
        num_inference_steps=num_inference_steps,  # Number of diffusion steps
        generator=torch.Generator("cuda").manual_seed(seed),  # Seed generator for reproducibility
    ).images

    # Displaying the prompt, seed, and the generated image
    print(f"Prompt:\t{prompt}\nSeed:\t{seed}")
    media.show_images(images)
    images[0].save("output.jpg")  # Saving the generated image

# Example prompts for the image generation demonstration
'''
Generate a photorealistic image in sepia tones of a poised woman from the early 20th century with a vintage portrait aesthetic. She should have refined features, a contemplative expression, and a hairstyle typical of the era. The woman is adorned in an elegant, draped gown with a wide neckline. Include subtly a second person's hands on her shoulder, adding a mysterious and layered effect. The image should be soft-focused with a vignette to draw attention to her face and expression, and have monochromatic tones to enhance the antique feel. Introduce a subtle grain to simulate photorealistic noise, typical of early film photography, creating a rich texture and depth. Ensure a high-quality render for a clear, crisp, and award-worthy photographic finish.

Generate a photorealistic image in sepia tones of a poised woman from the early 20th century with a vintage portrait aesthetic. She should have refined features and a contemplative expression, with particular attention to the clarity and detail in her eyes, ensuring they are well-defined and expressive. Her hairstyle should be typical of the era. The woman is adorned in an elegant, draped gown with a wide neckline. Subtly include a second person's hands on her shoulder, adding a mysterious and layered effect. While the overall image should be soft-focused with a vignette to draw attention to her face and expression, ensure that the eyes remain clear and sharp. Use monochromatic tones to enhance the antique feel but moderate the sepia effect and photorealistic noise around the eyes to maintain their clarity. The goal is to create a high-quality render for a clear, crisp, and award-worthy photographic finish, with special emphasis on realistic and detailed eyes.

'''


Generate a photorealistic image in sepia tones of a poised woman from the early 20th century with a vintage portrait aesthetic. She should have refined features, a contemplative expression, and a hairstyle typical of the era. The woman is adorned in an elegant, draped gown with a wide neckline. Include subtly a second person's hands on her shoulder, adding a mysterious and layered effect. The image should be soft-focused with a vignette to draw attention to her face and expression, and have monochromatic tones to enhance the antique feel. Introduce a subtle grain to simulate photorealistic noise, typical of early film photography, creating a rich texture and depth. Ensure a high-quality render for a clear, crisp, and award-worthy photographic finish.


Create a photorealistic image in sepia tones of a poised woman from the early 20th century, ensuring the portrait embodies a vintage aesthetic. Focus on rendering her features with refined detail, especially her eyes, which should be clear, well-defined, and expressive. Her hairstyle should reflect the era's fashion. She is elegantly dressed in a draped gown with a wide neckline. Subtly include another person's hands on her shoulder for a mysterious effect. The image should have a soft focus with a vignette to highlight her face and expression, but keep her eyes sharp and detailed. Use monochromatic sepia tones to enhance the antique feel. Add elements to simulate aging, like slight fading, mild discoloration, and subtle graininess throughout the image, akin to natural wear seen in old photographs. This noise should be balanced to ensure the overall quality remains high, with the woman's eyes as a clear, crisp focal point. The goal is a realistic, aged, award-worthy photographic finish with a blend of vintage charm and clarity in essential details.


Craft a photorealistic image in sepia tones portraying a dignified woman from the early 20th century, designed to replicate a vintage portrait style. Her features should be meticulously detailed, with a particular emphasis on her eyes, ensuring they are vivid, well-defined, and brimming with expression. The woman's hairstyle should authentically represent the fashion of that era. She is wearing a gracefully draped gown with a broad neckline. Include, with subtlety, another person's hands resting on her shoulder, adding a layer of intrigue and depth. The image should exhibit a soft-focus effect, with a vignette that directs attention to her face and particularly her eyes, which must remain sharp against the soft backdrop. Employ sepia tones to convey the antique atmosphere, but introduce varied aging effects such as slight creases, random speckles, mild fading, and a gentle unevenness in tone to mimic the wear and tear typical in old photographs. These elements of aging should be artfully balanced to preserve the high-quality, crisp nature of the image, particularly around the woman's facial features. The aim is to produce an award-worthy, photorealistic finish that skillfully blends the charm of vintage photography with the clarity and impact of modern photographic techniques.


Craft a photorealistic sepia-toned image that captures the essence of an early 20th-century woman in a vintage portrait style. Her features, especially her eyes, should be rendered with exceptional detail, radiating vividness and expression. The hairstyle should be true to the period's fashion. She's dressed in an elegantly draped gown with a wide neckline. Subtly include another individual's hands on her shoulder, lending an enigmatic quality to the portrait. The image should feature a soft-focus effect, with a vignette centering on her face, ensuring her eyes remain sharply detailed against the muted background. Apply sepia tones to evoke a timeless feel, and intersperse a diverse range of aging effects – including variable patterns of creasing, a mix of speckles, patches of fading, and tonal fluctuations – to authentically replicate the characteristics of an aged photograph. These aging effects should be strategically placed to add character while maintaining the high-quality, clear definition of the woman's facial features. The goal is to create an award-worthy, photorealistic finish that harmoniously blends the allure of vintage photography with the precision and impact of contemporary photo-rendering techniques.

Design a photorealistic sepia-toned image that vividly brings to life a woman from the early 20th century, captured in a vintage portrait style. Her facial features, particularly her eyes, should be exquisitely detailed, exuding depth and emotion. The hairstyle should elegantly reflect the fashion trends of that era. She is attired in a sophisticated, flowing gown with a wide neckline, and subtly include the gentle presence of another person's hands on her shoulder, adding a touch of mystery and narrative depth. The image should present a soft-focus effect, with a vignette that gracefully draws attention to her face, especially her eyes, which must stand out with sharp clarity. Implement sepia tones to infuse an antique ambiance, and integrate a diverse array of aging effects such as varied crease patterns, a blend of speckle sizes, uneven fading, and a dynamic range of tonal shifts to authentically simulate the wear of old photographs. These aging effects should be thoughtfully applied, enhancing the image's character while preserving the pristine definition of the woman's features. The aim is to create a photorealistic, award-worthy image that masterfully fuses the nostalgic charm of vintage photography with the nuanced clarity and sophistication of modern imaging techniques.

Compose a highly detailed, photorealistic sepia-toned image, capturing the dignified aura of a woman from the early 20th century in a classic portrait style. Focus on creating a vividly expressive face, particularly her eyes, which should be strikingly clear and full of life. Her hair should be styled in an era-appropriate manner, adding to her elegant demeanor. She is clad in a sophisticated gown with a distinctively wide neckline. Subtly incorporate the presence of another person's hands on her shoulder, introducing an element of intrigue and storytelling. The image should be softly focused, with a vignette effect that subtly highlights her face, especially the eyes, ensuring they contrast sharply with the softer elements. Use sepia tones to evoke a timeless, antique quality. Introduce a variety of aging effects, like a range of small creases, a scattering of speckles, areas of light and dark fading, and subtle tonal variations, to realistically depict the aging process of early photographs. These aging textures should be carefully balanced to maintain the integrity and sharpness of the woman's facial features, particularly her eyes. The final goal is to produce a photorealistic image that seamlessly blends the evocative beauty of vintage photography with the detailed sharpness and artistic sophistication of contemporary image rendering.

Generate a realistic sepia-toned image of a dignified gentleman from the Victorian era. He should have a distinguished appearance, with a neatly groomed beard and hair styled in a typical 19th-century fashion. He is wearing a formal three-piece suit with a waistcoat, a cravat or ascot tie, and a pocket watch. His expression should be contemplative and poised. The background should reflect a Victorian study, complete with bookshelves, a globe, and a writing desk. Add subtle aging effects to the image, such as mild fading, small creases, and a hint of grain, to mimic the look of an old photograph while keeping the gentleman's features sharply in focus.

Create a photorealistic image of a man from the 1920s, captured in a lively speakeasy setting. He should be wearing a classic pinstripe suit, indicative of the era, with a fedora hat and a thin mustache. His posture should exude confidence and charm. The speakeasy background should have an art deco style, with a jazz band playing and people dancing in the background. Introduce aging effects like slight sepia tones, soft speckles, and gentle fading around the edges to give the image an authentic 1920s feel